In [1]:
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np

import pandas as pd

In [2]:
train_pricing_decisions = pd.read_csv('../../data/train_prices_decisions.csv')
test_user_info = pd.read_csv('../../data/test_user_info.csv')

In [3]:
X = train_pricing_decisions[['Covariate1', 'Covariate2', 'Covariate3', 'price_item_0', 'price_item_1']]
y = train_pricing_decisions['item_bought'].map({-1: 2, 0:0, 1:1})

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
# from sklearn.ensemble import GradientBoostingClassifier
# clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=20, random_state=0).fit(X_train, y_train)
# clf.score(X_test, y_test)

In [6]:
import xgboost
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth' : [3, 4, 5],
    'min_child_weight' : [1,5,10],
    'learning_rate' : [0.01, 0.1, 0.2],
    'n_estimators' : [100, 200, 300]
}
clf = xgboost.XGBClassifier('multi:softmax')
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy', cv=5, verbose=1)

grid_search.fit(X_train, y_train)
print("Best Parameters found: ", grid_search.best_params_)
print("Best Parameters found: ", grid_search.best_score_)

clf = grid_search

/Users/cindy/anaconda3/lib/python3.11/site-packages/xgboost/core.py:617: FutureWarning: Pass `objective` as keyword args.
  warnings.warn(msg, FutureWarning)


Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best Parameters found:  {'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 200}
Best Parameters found:  0.9628358208955223


In [7]:
# y_pred = clf.predict(X_test)
# accuracy_score(y_test, y_pred)

## Generating prices for test users

In [8]:
# ## based on training dataset
# test_user_info_df = test_user_info[['Covariate1', 'Covariate2', 'Covariate3']]

# price_sequence = np.linspace(0, 100, 10)
# price_sequence_0 = np.linspace(50, 1000, 10)

# df_result = pd.DataFrame()

# for index, user in test_user_info_df.iterrows():
#     user_single = user.to_frame().T
#     user = pd.concat([user_single]*100, ignore_index=True)

#     for i in range(len(user)):
#         user.at[i, 'price_item_0'] = price_sequence_0[(i // len(price_sequence_0)) - 1]
#         user.at[i, 'price_item_1'] = price_sequence[i % len(price_sequence)]


#     probabilities = clf.predict_proba(user)

#     user[['p0', 'p1', 'p2']] = probabilities
#     user['revenue'] = user['price_item_0'] * user['p0'] + user['price_item_1'] * user['p1']

#     ## finding 
#     optimal_price_0 = user.loc[np.argmax(user['revenue']), 'price_item_0']
#     optimal_price_1 = user.loc[np.argmax(user['revenue']), 'price_item_1']

#     ## narrow down the range and find a better price
#     tmp = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
#     price_sequence_optimized_item_0 =  np.array([optimal_price_0]*11) + np.array(tmp)
#     price_sequence_optimized_item_1 =  np.array([optimal_price_1]*11) + np.array(tmp)

#     user_optimized = pd.concat([user_single]*100, ignore_index=True)
#     for i in range(len(user)):
#         user_optimized.at[i, 'price_item_0'] = price_sequence_optimized_item_0[(i // len(price_sequence_0)) - 1]
#         user_optimized.at[i, 'price_item_1'] = price_sequence_optimized_item_1[i % len(price_sequence)]

#     probabilities = clf.predict_proba(user_optimized)

#     user_optimized[['p0', 'p1', 'p2']] = probabilities
#     user_optimized['revenue'] = user_optimized['price_item_0'] * user_optimized['p0'] + user_optimized['price_item_1'] * user_optimized['p1']

#     # optimal_optimal price
#     optimal_optimal_price_0 = user_optimized.loc[np.argmax(user_optimized['revenue']), 'price_item_0']
#     optimal_optimal_price_1 = user_optimized.loc[np.argmax(user_optimized['revenue']), 'price_item_1']

#     expected_revenue = user_optimized.loc[np.argmax(user_optimized['revenue']), 'revenue']

#     df_result.loc[index, 'user_index'] = int(index + 30000)
#     df_result.loc[index, 'price_item_0'] = optimal_optimal_price_0
#     df_result.loc[index, 'price_item_1'] = optimal_optimal_price_1
#     df_result.loc[index, 'expected_revenue'] = expected_revenue

# df_result.set_index('user_index', inplace=True)
# # Convert the index to integer type
# df_result.index = df_result.index.astype(int)


# ### saving csv file
# df_result.to_csv('../part2_static_prices_submission.csv')


In [10]:
picklefile = open('trained_model', 'wb')
#pickle the object and store it in a file
pickle.dump(clf, picklefile)

In [11]:
#check that the object is correctly pickled and works when unpickled
del clf
picklefile = open('trained_model', 'rb')
new_model = pickle.load(picklefile)
# new_model.predict(np.array([1,2,3]).reshape(1,-1))